## Tune Vitrocal Parameters

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from vitrocal.datasets import catalog, ExcelDataset
from vitrocal.preprocessors import StandardPreprocessor
from vitrocal.detectors import DerivativeDetector, StandardExtractor
from vitrocal.analyzers import StandardAnalyzer

Pull out functions from `AnalyzeSingle.py` and work with them interactively

### Load example data file

In [ ]:
def load_data(fpath: str | os.PathLike, load_args: dict={}) -> pd.DataFrame:
    """Load single neuron output file.

    Args:
        fpath (str | os.PathLike): Path to single Excel spreadsheet.
        load_args (dict, optional): Passed to `pd.read_excel()`. Defaults to None.

    Returns:
        pd.DataFrame: Dataframe
    """
    fname = os.path.basename(fpath)
    dataset = ExcelDataset.ExcelDataset(fpath, load_args)
    return dataset.load(), fname

In [ ]:
df, fname = load_data("../../data/01_raw/E Green.xlsx")
df.head()

Note: you want to be super careful about loading your files and looking at them first.
These files don't have a header, and sometimes they have numbered rows like this one.


In [ ]:
load_args = {'header': None, 'index_col': 0}
df, fname = load_data("../../data/01_raw/E Green.xlsx", load_args)
df.head()

`Vitrocal` is organized into three distinct modules for data analysis:
* `vitrocal.preprocessors`
* `vitrocal.detectors`
* `vitrocal.analyzers`

You'll want to explore the parameter space for each of these individually and assess
the impacts different parameter combinations have on your data.

See [the documentation](https://ajbarrow.w3.uvm.edu/assets/public_share/vitrocal_docs/api/) for details.

### Preprocess

https://ajbarrow.w3.uvm.edu/assets/public_share/vitrocal_docs/api/#vitrocal.preprocessors.StandardPreprocessor   

In [ ]:
# change these
fps = 1/2.5
bleach_period = 60
filter_frequency = None
baseline_threshold = 10 # percent
preprocess_window_size = 60 # seconds

# instantiate the StandardPreprocess object with these parameters
preprocessor = StandardPreprocessor(
        frames_per_second=1/2.5,
        bleach_period=bleach_period,
        filter_frequency=filter_frequency,
        baseline_threshold=baseline_threshold,
        window_size=preprocess_window_size
)

In [ ]:
# call the object's `preprocess` method
# see the documentation for other methods available

processed = preprocessor.preprocess(df)

In [ ]:
processed.head()

## Detect and Extract

https://ajbarrow.w3.uvm.edu/assets/public_share/vitrocal_docs/api/#vitrocal.detectors.DerivativeDetector   

In [ ]:
# change this
threshold = 20 # percent
detector = DerivativeDetector(threshold)

detected = detector.detect(processed)
detected

Extract: https://ajbarrow.w3.uvm.edu/assets/public_share/vitrocal_docs/api/#vitrocal.detectors.StandardExtractor

Note: the `StandardExtractor` class has a `detect_and_extract` method

In [ ]:
# change these
window = (3, 30) # seconds before and after
threshold = 20 # percent

extractor = StandardExtractor(
    window=window,
    frames_per_second=fps, # defined above
    threshold=threshold
)

events = extractor.extract(processed, detected)
events

Note: this method returns events as a Python dictionary. You can examine individual events, if you like:

In [ ]:
roi = 2
events[roi]

## Analyze
https://ajbarrow.w3.uvm.edu/assets/public_share/vitrocal_docs/api/#vitrocal.analyzers.StandardAnalyzer

In [ ]:
# change these
upper_decay_bound = 0.8 # proportion
lower_decay_bound = 0.2 # proportion

analyzer = StandardAnalyzer(
    upper_decay_bound=upper_decay_bound,
    lower_decay_bound=lower_decay_bound
)

result, avg_result = analyzer.analyze(events)

In [ ]:
result

In [ ]:
avg_result

## Plotting

Jupyter notebooks are very helpful for making quick plots to confirm your intuition about an analysis problem. The syntax can be tricky: https://matplotlib.org

[Seaborn](https://seaborn.pydata.org) can make life a little easier, but you generally need to work with `pandas.DataFrames` for the input data.

In [ ]:
N = len(result)

plt.scatter(np.repeat('peak', N), result['peak'])
plt.scatter(np.repeat('upper', N), result['upper'])
plt.scatter(np.repeat('lower', N), result['lower'])